In [ ]:
'''Train a model for the CY-metric'''

In [1]:
# Generate training data
import numpy as np
import os as os
import pickle as pickle
from cymetric.pointgen.pointgen import PointGenerator

monomials = 5*np.eye(5, dtype=np.int64)
coefficients = np.ones(5)
kmoduli = np.ones(1)
ambient = np.array([4])

pg = PointGenerator(monomials, coefficients, kmoduli, ambient)
points = pg.generate_points(100)
pg.cy_condition(points)

dirname = 'cy_models/train_data'
n_p = 100000
kappa = pg.prepare_dataset(n_p, dirname)
data = np.load(os.path.join(dirname, 'dataset.npz'))
pg.prepare_basis(dirname, kappa=kappa)


/Users/ed/miniforge3/envs/cymetric/lib/python3.9/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/ed/miniforge3/envs/cymetric/lib/python3.9/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
pointgen:INFO:Vol_k: 5.000000000000001, Vol_cy: 284.2561104432773.


0

In [2]:
import tensorflow as tf
tfk = tf.keras
from cymetric.models.callbacks import SigmaCallback
from cymetric.models.tfmodels import PhiFSModel
from cymetric.models.metrics import SigmaLoss
from cymetric.models.tfhelper import prepare_tf_basis, train_model

data = np.load(os.path.join(dirname, 'dataset.npz'))
BASIS = np.load(os.path.join(dirname, 'basis.pickle'), allow_pickle=True)
BASIS = prepare_tf_basis(BASIS)

n_out = 1
nlayer = 3
nHidden = 64
act = 'gelu'
nEpochs = 10
bSizes = [64, 50000]
alpha = [1., 1., 1., 1., 1.]
nfold = 3
n_in = 2*5

nn_phi = tf.keras.Sequential()
nn_phi.add(tfk.Input(shape=(n_in)))
for i in range(nlayer):
    nn_phi.add(tfk.layers.Dense(nHidden, activation=act))
nn_phi.add(tfk.layers.Dense(n_out, use_bias=False))
from cymetric.models.tfmodels import PhiFSModel
phimodel = PhiFSModel(nn_phi, BASIS, alpha=alpha)
opt_phi = tfk.optimizers.Adam()
scb = SigmaCallback((data['X_val'], data['y_val']))
cb_list = [scb]
cmetrics = [SigmaLoss()]

phimodel, training_history = train_model(phimodel, data, optimizer=opt_phi, epochs=nEpochs, batch_sizes=bSizes, 
                                       verbose=1, custom_metrics=cmetrics, callbacks=cb_list)


absl:WARNING:At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.



Epoch  1/10
2/2 [==============================] - 7s 2s/step - sigma_loss: 0.1471 - sigma_val: 0.1558

Epoch  2/10
2/2 [==============================] - 4s 2s/step - sigma_loss: 0.1530 - sigma_val: 0.1555

Epoch  3/10
2/2 [==============================] - 4s 2s/step - sigma_loss: 0.1370 - sigma_val: 0.1491

Epoch  4/10
2/2 [==============================] - 4s 2s/step - sigma_loss: 0.1356 - sigma_val: 0.1440

Epoch  5/10
2/2 [==============================] - 4s 2s/step - sigma_loss: 0.1253 - sigma_val: 0.1338

Epoch  6/10
2/2 [==============================] - 4s 2s/step - sigma_loss: 0.0918 - sigma_val: 0.0938

Epoch  7/10
2/2 [==============================] - 4s 2s/step - sigma_loss: 0.0645 - sigma_val: 0.0622

Epoch  8/10
2/2 [==============================] - 4s 2s/step - sigma_loss: 0.0537 - sigma_val: 0.0514

Epoch  9/10
2/2 [==============================] - 4s 2s/step - sigma_loss: 0.0423 - sigma_val: 0.0408

Epoch 10/10
2/2 [==============================] - 4s 2s/step -

In [3]:
#Save the trained model
save_filepath = '/Users/ed/Documents/Projects/G2/Flows/G2Metric/github/cy_models/test_model.keras'
phimodel.save(save_filepath)


In [4]:
#Import the trained model
imported_model = PhiFSModel(nn_phi, BASIS, alpha=alpha)
imported_model.nn_phi = tfk.models.load_model(save_filepath)
